# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import matplotlib.pyplot as plt #The matplotlib visuailzation library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'week52change': 1.225665,
 'week52high': 461.32,
 'week52low': 204.57,
 'marketcap': 1983637012036,
 'employees': 137270,
 'day200MovingAvg': 308.06,
 'day50MovingAvg': 388,
 'float': 4416173802,
 'avg10Volume': 46795125.8,
 'avg30Volume': 38014343.6,
 'ttmEPS': 13.7858,
 'ttmDividendRate': 3.19,
 'companyName': 'Apple, Inc.',
 'sharesOutstanding': 4279254023,
 'maxChangePercent': 448.595,
 'year5ChangePercent': 2.889,
 'year2ChangePercent': 1.177,
 'year1ChangePercent': 1.235,
 'ytdChangePercent': 0.5054,
 'month6ChangePercent': 0.4076,
 'month3ChangePercent': 0.4835,
 'month1ChangePercent': 0.1948,
 'day30ChangePercent': 0.2622,
 'day5ChangePercent': 0.02,
 'nextDividendDate': '2020-08-26',
 'dividendYield': 0.007338304768454052,
 'nextEarningsDate': '2020-10-22',
 'exDividendDate': '2020-08-04',
 'peRatio': 33.48,
 'beta': 1.150515780358486}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course.

In [4]:
data['year1ChangePercent']

1.235

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,101.75,0.449500,N/A
1,AAL,13.21,-0.570900,N/A
2,AAP,156.32,0.055900,N/A
3,AAPL,460.11,1.280000,N/A
4,ABBV,94.62,0.447700,N/A
...,...,...,...,...
500,YUM,92.53,-0.233944,N/A
501,ZBH,140.12,0.003324,N/A
502,ZBRA,286.56,0.344929,N/A
503,ZION,35.17,-0.201861,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DXCM,442.00,1.867137,N/A
1,NVDA,457.11,1.863956,N/A
2,AMD,88.32,1.559064,N/A
3,CARR,29.12,1.455054,N/A
4,AAPL,460.11,1.280000,N/A
5,REGN,649.29,1.037491,N/A
6,WST,274.83,0.953620,N/A
7,PYPL,203.98,0.863173,N/A
8,LRCX,385.86,0.853481,N/A
9,QRVO,135.31,0.818240,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy.

Since we've already done this, try to complete the following two code cells without watching me do it first!

In [23]:
def portfolio_input():
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
100000


In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Number Of Shares to Buy
0,DXCM,442.00,1.867137,N/A,4.0
1,NVDA,457.11,1.863956,N/A,4.0
2,AMD,88.32,1.559064,N/A,22.0
3,CARR,29.12,1.455054,N/A,67.0
4,AAPL,460.11,1.280000,N/A,4.0
5,REGN,649.29,1.037491,N/A,3.0
6,WST,274.83,0.953620,N/A,7.0
7,PYPL,203.98,0.863173,N/A,9.0
8,LRCX,385.86,0.853481,N/A,5.0
9,QRVO,135.31,0.818240,N/A,14.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe.

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])


for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])
    
hqm_dataframe

0      88.9109
1      2.37624
2      54.8515
3      99.2079
4      89.3069
        ...   
500    22.7723
501    47.3267
502    84.5545
503    24.9505
504    79.0099
Name: One-Year Return Percentile, Length: 505, dtype: object
0      86.1386
1      1.78218
2      82.1782
3      95.6436
4      64.3564
        ...   
500    41.5842
501    40.7921
502    81.3861
503    22.9703
504    78.6139
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      78.6139
1      92.6733
2      81.9802
3      97.2277
4      44.1584
        ...   
500    28.9109
501    55.0495
502    62.7723
503    43.3663
504    82.5743
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      66.3366
1      64.5545
2      74.2574
3      93.8614
4      4.15842
        ...   
500    37.8218
501    91.4851
502    68.7129
503    48.7129
504    85.5446
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,100.66,N/A,0.451200,88.9109,0.180300,86.1386,0.247800,78.6139,0.097500,66.3366,N/A
1,AAL,13.47,N/A,-0.565700,2.37624,-0.562200,1.78218,0.373300,92.6733,0.095000,64.5545,N/A
2,AAP,158.12,N/A,0.057700,54.8515,0.147900,82.1782,0.273000,81.9802,0.116800,74.2574,N/A
3,AAPL,454.06,N/A,1.261000,99.2079,0.395100,95.6436,0.476900,97.2277,0.193300,93.8614,N/A
4,ABBV,95.60,N/A,0.453900,89.3069,0.007100,64.3564,0.104000,44.1584,-0.066800,4.15842,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,92.65,N/A,-0.225979,22.7723,-0.122530,41.5842,0.054279,28.9109,0.050072,37.8218,N/A
501,ZBH,141.78,N/A,0.003295,47.3267,-0.124119,40.7921,0.145037,55.0495,0.179935,91.4851,N/A
502,ZBRA,288.22,N/A,0.351161,84.5545,0.143335,81.3861,0.168856,62.7723,0.101728,68.7129,N/A
503,ZION,34.55,N/A,-0.207590,24.9505,-0.263508,22.9703,0.103661,43.3663,0.069402,48.7129,N/A


## Calculating the HQM Score

In [21]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
31,AMD,87.44,N/A,1.563700,99.604,0.644219,98.8119,0.619330,99.2079,0.598321,99.802,99.3564
78,CARR,29.23,N/A,1.448092,99.4059,1.452131,100,0.597568,98.8119,0.209514,95.2475,98.3663
3,AAPL,454.06,N/A,1.261000,99.2079,0.395100,95.6436,0.476900,97.2277,0.193300,93.8614,96.4851
6,ABMD,320.18,N/A,0.648200,96.4356,0.683200,99.2079,0.678500,99.4059,0.177900,90.8911,96.4851
490,WST,284.00,N/A,0.945184,98.8119,0.719141,99.604,0.375078,92.8713,0.185106,92.6733,95.9901
...,...,...,...,...,...,...,...,...,...,...,...,...
84,CCL,14.31,N/A,-0.720321,0.39604,-0.664216,0.594059,0.000717,17.0297,-0.075056,2.9703,5.24752
181,FE,30.50,N/A,-0.341441,11.2871,-0.434234,6.73267,-0.286663,0.19802,-0.275813,0.19802,4.60396
479,WDC,39.00,N/A,-0.345079,10.495,-0.477939,4.75248,-0.166001,1.58416,-0.137913,0.594059,4.35644
219,HFC,26.00,N/A,-0.511784,3.9604,-0.431497,6.93069,-0.184098,1.38614,-0.078492,2.77228,3.76238


## Selecting the 50 Best Momentum Stocks

In [ ]:
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

In [ ]:
portfolio_input()

In [26]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number Of Shares to Buy
0,A,100.66,1,0.451200,88.9109,0.180300,86.1386,0.247800,78.6139,0.097500,66.3366,80,1.0
1,AAL,13.47,14,-0.565700,2.37624,-0.562200,1.78218,0.373300,92.6733,0.095000,64.5545,40.3465,14.0
2,AAP,158.12,1,0.057700,54.8515,0.147900,82.1782,0.273000,81.9802,0.116800,74.2574,73.3168,1.0
3,AAPL,454.06,0,1.261000,99.2079,0.395100,95.6436,0.476900,97.2277,0.193300,93.8614,96.4851,0.0
4,ABBV,95.60,2,0.453900,89.3069,0.007100,64.3564,0.104000,44.1584,-0.066800,4.15842,50.495,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,92.65,2,-0.225979,22.7723,-0.122530,41.5842,0.054279,28.9109,0.050072,37.8218,32.7723,2.0
501,ZBH,141.78,1,0.003295,47.3267,-0.124119,40.7921,0.145037,55.0495,0.179935,91.4851,58.6634,1.0
502,ZBRA,288.22,0,0.351161,84.5545,0.143335,81.3861,0.168856,62.7723,0.101728,68.7129,74.3564,0.0
503,ZION,34.55,5,-0.207590,24.9505,-0.263508,22.9703,0.103661,43.3663,0.069402,48.7129,35,5.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)